<a href="https://colab.research.google.com/github/longjae/raft_rag/blob/master/negative_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install FlagEmbedding==1.3.3 peft
!wget https://github.com/KLUE-benchmark/KLUE/raw/main/klue_benchmark/klue-mrc-v1.1/klue-mrc-v1.1_train.json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.8/161.8 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━

In [2]:
import json
import pandas as pd
from tqdm import tqdm
from FlagEmbedding import BGEM3FlagModel
import numpy as np
import datasets
from huggingface_hub import login
from datasets import load_dataset

In [3]:
with open("klue-mrc-v1.1_train.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 필요한 정보 추출
records = []
for article in data["data"]:
    title = article["title"]
    news_category = article.get("news_category", "N/A")
    source = article.get("source", "N/A")

    for paragraph in article["paragraphs"]:
        context = paragraph["context"]

        for qa in paragraph["qas"]:
            question = qa["question"]
            question_type = qa["question_type"]
            is_impossible = qa["is_impossible"]

            for answer in qa["answers"]:
                answer_text = answer["text"]
                answer_start = answer["answer_start"]

                records.append(
                    {
                        "title": title,
                        "news_category": news_category,
                        "source": source,
                        "context": context,
                        "question": question,
                        "question_type": question_type,
                        "is_impossible": is_impossible,
                        "answer_text": answer_text,
                        "answer_start": answer_start,
                    }
                )

# DataFrame으로 변환
df = pd.DataFrame(records)

In [4]:
# 질문 기준으로 중복 샘플 제거
df = df.drop_duplicates(subset="question")

In [5]:
# 본문 기준으로 중복 샘플 제거
df = df.drop_duplicates(subset="context")

In [6]:
# 질문, 본문, 정답
df[["context", "question", "answer_text"]]

,context,question,answer_text
0,올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도...,북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?,한 달가량
2,부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...,지능형 생산자동화 기반기술을 개발중인 스타트업은?,삼보테크놀로지
3,시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인으...,개막전에서 3안타 2실점을 기록해서 패한 선수는?,와쿠이 히데아키
4,유명 맛집 이름을 달고 나온 편의점 자체상표(PB) 라면이 인기를 끌고 있다. ‘검...,컵라면 매출에서 불닭볶음면을 이긴 상품은?,‘교동반점 짬뽕’
8,“앞으로 5년 안에 아시아 친환경·신재생에너지 투자의 황금기가 도래할 것입니다.”기...,정부에게 환경과 관련해서 우선적으로 원조 받고 있는 곳은?,환경오염이 심한 지역
...,...,...,...
17656,“오너 경영은 중견기업이 글로벌 전문기업이나 대기업으로 성장하는 밑거름이 될 것입니...,이동기 학회장은 오너 경영을 무엇과 비교했는가?,전문 경영인 체제
17657,홈 뷰티 디바이스 브랜드 ‘엘리닉(L.linic)’이 고객 감사의 달을 맞아 전국 ...,팝업스토어 앵콜 행사를 가장 먼저 시작하는 곳은 어디인가?,롯데백화점 잠실점
17658,유인원 무리의 리더 시저는 인간 건축가 말콤(제이슨 클락)에게 작별 인사를 한다. ...,혹성탈출의 두 번째 프리퀄의 제목은?,‘혹성탈출: 반격의 서막’
17660,ASUS(에이수스) 그래픽카드 공식수입사 인텍앤컴퍼니(대표 서정욱)는 10월 16일...,인택엔컴퍼니가 실시하는 추첨판매 신청 마감시간은?,8시 10분


In [7]:
# 임베딩 모델 로드
model = BGEM3FlagModel("BAAI/bge-m3", use_fp16=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

...

sparse_linear.pt:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

In [8]:
# 문장을 임베딩으로 변환하는 함수
def get_embeddings(sentences, batch_size=32):
    return model.encode(sentences, batch_size=batch_size, max_length=512)["dense_vecs"]

In [9]:
# 최대 32개의 데이터를 동시에 임베딩하는 함수. 훨씬 빠르게 임베딩하기 위해 배치(데이터 묶음)로 처리합니다.
embeddings = get_embeddings(["안녕하세요", "반갑습니다"])
print("임베딩 벡터의 개수:", len(embeddings))

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


임베딩 벡터의 개수: 2


In [10]:
# 본문 전체를 임베딩
print("Generating context embeddings...")
context_embeddings = get_embeddings(df["context"].tolist())

Generating context embeddings...


Inference Embeddings: 100%|██████████| 327/327 [04:32<00:00,  1.20it/s]


In [11]:
# negative_samples 열을 추가할 리스트
negative_samples_list = []

# 각 질문(question)에 대해 유사도가 높은 4개의 다른 본문(context)를 찾기
print("Finding negative samples...")
for i in tqdm(range(len(df)), total=len(df), dynamic_ncols=True):
    question = df.iloc[i]["question"]

    # 질문 임베딩 생성
    question_embedding = get_embeddings([question])[0]

    # 코사인 유사도 계산
    similarities = np.dot(context_embeddings, question_embedding)

    # 현재 문서의 인덱스를 제외한 4개의 문서를 선택
    top_n_indices = similarities.argsort()[::-1]
    top_n_indices = [idx for idx in top_n_indices if idx != i][
        :4
    ]  # 현재 문서 제외, 상위 4개 선택

    # 선택된 문서들을 negative_samples 리스트로 추가
    negative_samples = [df["context"].iloc[idx] for idx in top_n_indices]
    negative_samples_list.append(negative_samples)

Finding negative samples...


100%|██████████| 10434/10434 [18:58<00:00,  9.17it/s]


In [12]:
# 결과를 새로운 열로 추가. 각 질문에 대해서 전체 본문(10,434개)과 유사도 비교를 했을때 유사도가 높은 본문 4개가 추가된다.
df["negative_samples"] = negative_samples_list

# 결과 확인
print(df.head())  # 결과의 처음 5개 행을 출력

                                         title news_category     source  \
0                       제주도 장마 시작 … 중부는 이달 말부터            종합   hankyung   
2      부산정보산업진흥원, 과기부 지역SW서비스사업화 지원사업 4개 과제 선정            경제    acrofan   
3                                     나루세 요시히사          None  wikipedia   
4                               편의점 휩쓴 ‘맛집 라면’          생활경제   hankyung   
8  사모펀드 ACP 아난드 프라카시 전무 ...“아시아 친환경 투자 황금기 온다”            기획   hankyung   

                                             context  \
0  올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도...   
2  부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...   
3  시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인으...   
4  유명 맛집 이름을 달고 나온 편의점 자체상표(PB) 라면이 인기를 끌고 있다. ‘검...   
8  “앞으로 5년 안에 아시아 친환경·신재생에너지 투자의 황금기가 도래할 것입니다.”기...   

                              question  question_type  is_impossible  \
0  북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?              1          False   
2          지능형 생산자동화 기반기술을 개발중인 스타트업은?              2          False

In [13]:
df = df.reset_index(drop=True)

In [14]:
idx = 5

print("질문 :", df["question"].loc[idx])
print("정답 문서:", df["context"].loc[idx])
print("네거티브 문서:", df["negative_samples"].loc[idx])
print("네거티브 문서의 개수:", len(df["negative_samples"].loc[idx]))

질문 : 에티하드 웰니스 프로그램의 일환으로 위생에 관한 정보를 제공하는 것은 누구인가?
정답 문서: 아랍 에미리트의 국영항공사 에티하드항공이 지난 10월 21일 열린 ’2020비즈니스 트래블러 중동 어워드(Business Traveller Middle East Awards 2020)에서 ‘최고의 이코노미 클래스(Best Economy Class)’와 ‘최고의 상용고객 프로그램(Best Frequent Flyer Programme’)’상의 영예를 안았다. 이는 2012년과 2013년에 열린 비즈니스 트래블러 중동 어워드에서 각각 최고의 이코노미 클래스, 최고의 상용 고객프로그램 분야의 첫 수상을 한 이래로 2회째 수상 기록이다. 지난 해, 에티하드항공은 다양한 수상 경력을 지닌 비즈니스, 퍼스트, 더 레지던스 캐빈에서 펼쳐 온 혁신과 마찬가지로 이코노미석에서도 기내식 선택권을 넓히고 무선 기내 엔터테인먼트, 인체공학적 디자인으로 설계된 좌석과 여유로운 레그룸을 가진 이코노미석 도입을 비롯해 옆 좌석 비우기, 보다 넉넉한 이코노미 스페이스 석 선택 옵션, 선호좌석 선택 사항을 제공하는 등 기내 경험에 대한 더욱 다양한 선택권을 제공해 왔다. 이를 통해 승객이 개인의 필요와 요구에 맞는 맞춤화된 선택을 할 수 있도록 돕고 만족도 높은 항공 여행을 할 수 있도록 서비스와 상품 개선에 투자를 지속해 왔다. 또한, 에티하드항공은 항공사의 건강 및 위생 프로그램인 ‘에티하드 웰니스’프로그램을 통해 신종코로나바이러스로 인한 전례 없는 시기에 필수적인 여행을 해야만 하는 여행자들에게 마음의 평화와 여행에 대한 자신감을 심어주기 위해 펼친 다양한 노력이 고객의 긍정적인 평가를 받았다고 전했다. 예약 과정에서부터 공항 이용은 물론 항공여행에 이르기까지 인공지능 기술을 비롯한 최신 기술을 과감히 도입해 광범위한 예방 조치를 시행하고 있으며 업계 최초로 선보인 에티하드 웰니스 프로그램에서는 특별 훈련 과정을 거친 에티하드항공의 웰니스 엠버서더가 여행 전 과정에 걸친 조언과 건강 및

In [22]:
from google.colab import userdata
HF_API_KEY = userdata.get("HF_WRITE_API_KEY")
login(HF_API_KEY)

In [23]:
dataset = datasets.Dataset.from_pandas(df)

In [24]:
dataset.push_to_hub("klue-mrc-bge-m3")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/longjae/klue-mrc-bge-m3/commit/62b186acc8f36fd563043f3b98a6f6d60933a418', commit_message='Upload dataset', commit_description='', oid='62b186acc8f36fd563043f3b98a6f6d60933a418', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/longjae/klue-mrc-bge-m3', endpoint='https://huggingface.co', repo_type='dataset', repo_id='longjae/klue-mrc-bge-m3'), pr_revision=None, pr_num=None)

In [25]:
# 1. Hugging Face에서 데이터셋 불러오기
dataset = load_dataset("longjae/klue-mrc-bge-m3")

# 2. train split을 데이터프레임으로 변환
df = dataset["train"].to_pandas()

# 3. 데이터프레임 확인
print(df.head())

Generating train split:   0%|          | 0/10434 [00:00<?, ? examples/s]

                                         title news_category     source  \
0                       제주도 장마 시작 … 중부는 이달 말부터            종합   hankyung   
1      부산정보산업진흥원, 과기부 지역SW서비스사업화 지원사업 4개 과제 선정            경제    acrofan   
2                                     나루세 요시히사          None  wikipedia   
3                               편의점 휩쓴 ‘맛집 라면’          생활경제   hankyung   
4  사모펀드 ACP 아난드 프라카시 전무 ...“아시아 친환경 투자 황금기 온다”            기획   hankyung   

                                             context  \
0  올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도...   
1  부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...   
2  시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인으...   
3  유명 맛집 이름을 달고 나온 편의점 자체상표(PB) 라면이 인기를 끌고 있다. ‘검...   
4  “앞으로 5년 안에 아시아 친환경·신재생에너지 투자의 황금기가 도래할 것입니다.”기...   

                              question  question_type  is_impossible  \
0  북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?              1          False   
1          지능형 생산자동화 기반기술을 개발중인 스타트업은?              2          False

In [26]:
df

,title,news_category,source,context,question,question_type,is_impossible,answer_text,answer_start,negative_samples
0,제주도 장마 시작 … 중부는 이달 말부터,종합,hankyung,올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도...,북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?,1,False,한 달가량,478,[궤도물리학은 계절의 지속 기간이 지구의 궤도가 지점과 분점 사이의 공간을 휩쓸고 ...
1,"부산정보산업진흥원, 과기부 지역SW서비스사업화 지원사업 4개 과제 선정",경제,acrofan,부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...,지능형 생산자동화 기반기술을 개발중인 스타트업은?,2,False,삼보테크놀로지,1422,"[IIoT(Industry Internet of Thing), 산업현장 스마트 팩토..."
2,나루세 요시히사,None,wikipedia,시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인으...,개막전에서 3안타 2실점을 기록해서 패한 선수는?,2,False,와쿠이 히데아키,107,[1960년에 주니치에 입단하여 같은 해 1960년 5월 7일 다이요 웨일스전에서 ...
3,편의점 휩쓴 ‘맛집 라면’,생활경제,hankyung,유명 맛집 이름을 달고 나온 편의점 자체상표(PB) 라면이 인기를 끌고 있다. ‘검...,컵라면 매출에서 불닭볶음면을 이긴 상품은?,2,False,‘교동반점 짬뽕’,408,[‘짜파구리’로 시작된 ‘나만의 레시피’ 열풍이 올해 식품시장을 주도한 것으로 나타...
4,사모펀드 ACP 아난드 프라카시 전무 ...“아시아 친환경 투자 황금기 온다”,기획,hankyung,“앞으로 5년 안에 아시아 친환경·신재생에너지 투자의 황금기가 도래할 것입니다.”기...,정부에게 환경과 관련해서 우선적으로 원조 받고 있는 곳은?,1,False,환경오염이 심한 지역,661,[포브스는 최근 “석탄은 기원전 315년 그리스 문헌에 대장간에서 원료로 쓰던 기록...
...,...,...,...,...,...,...,...,...,...,...
10429,인터뷰/ 이동기 신임 중견기업학회장,중기 의료,hankyung,“오너 경영은 중견기업이 글로벌 전문기업이나 대기업으로 성장하는 밑거름이 될 것입니...,이동기 학회장은 오너 경영을 무엇과 비교했는가?,1,False,전문 경영인 체제,229,[금융회사 지배구조 개선을 위해 은행과 자산총액 2조원 이상인 보험사와 증권사는 이...
10430,"엘리닉, 고객 감사의 달 맞아 팝업스토어 앵콜 오픈",산업,acrofan,홈 뷰티 디바이스 브랜드 ‘엘리닉(L.linic)’이 고객 감사의 달을 맞아 전국 ...,팝업스토어 앵콜 행사를 가장 먼저 시작하는 곳은 어디인가?,1,False,롯데백화점 잠실점,134,[뉴욕관광청은 뉴욕이 가장 화려하게 빛나는 계절인 겨울에 뉴욕 여행을 계획하는 여행...
10431,유인원과 인간의 치열한 생존 전쟁,문화/TV,hankyung,유인원 무리의 리더 시저는 인간 건축가 말콤(제이슨 클락)에게 작별 인사를 한다. ...,혹성탈출의 두 번째 프리퀄의 제목은?,2,False,‘혹성탈출: 반격의 서막’,170,[아오모리현의 핵연료 재처리 시설에서 의문의 두 사람에 의해 물체가 강탈된다. 강탈...
10432,"인텍앤컴퍼니, ASUS ROG STRIX GeForce RTX3080 OC 국내 출...",테크,acrofan,ASUS(에이수스) 그래픽카드 공식수입사 인텍앤컴퍼니(대표 서정욱)는 10월 16일...,인택엔컴퍼니가 실시하는 추첨판매 신청 마감시간은?,1,False,8시 10분,197,[법정관리(기업회생절차) 중인 팬택 매각의 본입찰 일정이 연기됐다. 인수의향서(LO...
